# Call the Azure Function from Fabric Notebook

This notebook demonstrates how to call the Hub Function from a **Fabric interactive notebook session** to get a Service Principal token.

## How it Works (Easy!)

1. **Get user token** - Fabric automatically provides your interactive user's token
2. **Call Function** - Send POST request with token to get SP token
3. **Use SP token** - Get the Service Principal token for downstream operations

## Setup (One-time)

Before running this notebook:
1. Deploy the Function App to Azure
2. Update the configuration values below with your actual values:
   - `FUNC_APP_CLIENT_ID` - From your deployed Function App
   - `FUNCTION_URL` - Your Function App URL (or MPE private endpoint)

## Notes

- ✓ Works in Fabric notebook sessions (interactive user runs)
- ✓ Automatically handles authentication via your Fabric credentials
- ✓ Uses MPE (Managed Private Endpoints) for secure calls when configured
- ✓ Returns both SP token and caller audit information

In [ ]:
from notebookutils import mssparkutils
import requests
import json

# ═══════════════════════════════════════════════════════════════
# CONFIGURATION - Update these values for your deployment
# ═══════════════════════════════════════════════════════════════

# Your Function App's Client ID (from Azure Portal or deployment)
FUNC_APP_CLIENT_ID = "1dbe7604-4c66-4d06-859e-06230744526c"

# Your Function App URL
# For public endpoint: https://<func-app-name>.azurewebsites.net/api/GetSPToken
# For MPE private endpoint: https://<private-fqdn>/api/GetSPToken
FUNCTION_URL = "https://your-function-app.azurewebsites.net/api/GetSPToken"

# Target scope for the Service Principal token
TARGET_SCOPE = "https://management.azure.com/.default"

# ═══════════════════════════════════════════════════════════════

print("🔹 Step 1: Getting your user token from Fabric...")

try:
    # Get your interactive user's token (automatic in notebook sessions)
    audience = f"api://{FUNC_APP_CLIENT_ID}"
    token = mssparkutils.credentials.getToken(audience)
    print(f"✓ User token acquired ({len(token)} chars)")
except Exception as e:
    print(f"✗ Failed to get token: {e}")
    print("  Note: This must run in a Fabric notebook session, not a pipeline")
    raise

print("\n🔹 Step 2: Calling the Hub Function...")

try:
    # Prepare the request
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "targetScope": TARGET_SCOPE
    }
    
    # Call the Function App
    print(f"  POST {FUNCTION_URL}")
    response = requests.post(FUNCTION_URL, headers=headers, json=payload, timeout=30)
    
    print(f"✓ Response received: {response.status_code}")
    
except requests.exceptions.ConnectionError as e:
    print(f"✗ Connection failed: {e}")
    print("  Check that the Function URL is correct and accessible")
    raise
except Exception as e:
    print(f"✗ Error: {e}")
    raise

print("\n🔹 Step 3: Processing the response...")

if response.status_code == 200:
    body = response.json()
    
    # Extract SP token info
    sp_token = body.get("access_token")
    caller = body.get("caller", {})
    
    print("\n✓ SUCCESS! Service Principal token obtained:\n")
    print(f"  Token Type: {body.get('token_type')}")
    print(f"  Expires In: {body.get('expires_in')} seconds ({body.get('expires_in')//3600} hour(s))")
    print(f"  Scope: {body.get('scope')}")
    
    print(f"\n✓ Caller Information:")
    print(f"  Type: {caller.get('type')} {'(Interactive User)' if caller.get('type') == 'USER' else '(Pipeline/MSI)'}")
    print(f"  Identity: {caller.get('display')}")
    print(f"  OID: {caller.get('oid')}")
    print(f"  Is Pipeline: {caller.get('is_pipeline')}")
    
    print(f"\n✓ Token preview: {sp_token[:50]}...")
    print(f"\n💾 SP Token ready to use! Store in a variable if needed:")
    print(f"  sp_token = '{sp_token[:30]}...'")
    
else:
    print(f"✗ Error ({response.status_code}):")
    try:
        error_data = response.json()
        print(json.dumps(error_data, indent=2))
    except:
        print(response.text)

In [ ]:
# Store the SP token for use in downstream operations
if response.status_code == 200:
    sp_token_data = response.json()
    sp_access_token = sp_token_data.get("access_token")
    
    print("=" * 60)
    print("Using the Service Principal Token")
    print("=" * 60 + "\n")
    
    # Example 1: Use with Azure SDK (Azure libraries)
    print("📌 Example 1: Using with Azure SDK\n")
    print("from azure.identity import ClientSecretCredential")
    print("# Use sp_access_token directly or refresh via SDK\n")
    
    # Example 2: Use with requests for REST API calls
    print("📌 Example 2: Using with REST API calls\n")
    print("headers = {'Authorization': f'Bearer {sp_access_token}'}")
    print("response = requests.get('https://management.azure.com/subscriptions/...', headers=headers)\n")
    
    # Example 3: Store for reuse in same notebook
    print("📌 Example 3: Reusing in this notebook\n")
    print(f"# Token stored in variable: sp_access_token")
    print(f"# Type: {type(sp_access_token)}")
    print(f"# Length: {len(sp_access_token)} characters")
    print(f"# Expires in: {sp_token_data.get('expires_in')} seconds\n")
    
    print("✓ Ready to use sp_access_token in downstream cells!")
else:
    print("❌ Cannot use token - API returned error")

## Troubleshooting

| Issue | Solution |
|-------|----------|
| `mssparkutils not found` | You're not in a Fabric notebook. Use `from notebookutils import mssparkutils` in Fabric |
| 401 Unauthorized | Function App client ID or token is wrong. Check FUNC_APP_CLIENT_ID matches deployment |
| 403 Forbidden | Your MSI/identity is not in the whitelist. Add your OID to `allowed-msi-oids` in Key Vault |
| Connection timeout | Function URL might be wrong or private endpoint not accessible from your network |
| 500 Internal Error | Check Function App logs. Key Vault secrets might be misconfigured |